In [26]:
import pandas as pd
import emoji
import emoji_vietnamese
import string

## Table of Contents
- [Load data](#load-data)
- [Remove #ERROR! values](#remove-error-values)
- [Remove duplicates](#remove-duplicates)
- [Remove outliers](#remove-outliers)
- [Remove special characters](#remove-special-characters)
  - [URL](#remove-url)
  - [Punctuation](#remove-punctuation)
  - [Stopwords](#remove-stopwords)
- [Lowercasing](#lowercasing)
- [Emoji processing](#emoji-processing)
- [Save cleaned data](#save-cleaned-data)

## Load data

In [27]:
train = pd.read_csv('../Dataset/vihsd/train.csv')

In [28]:
train.head()

,free_text,label_id
0,Em được làm fan cứng luôn rồi nè ❤️ reaction q...,0
1,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,2
2,Đậu Văn Cường giờ giống thằng sida hơn à,0
3,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN. NHÀ NƯ...,2
4,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,0


In [29]:
print(train['label_id'].value_counts())

label_id
0    19886
2     2556
1     1606
Name: count, dtype: int64


## Remove null values

In [30]:
train.isnull().sum()

free_text    2
label_id     0
dtype: int64

In [31]:
train = train.dropna()

## Remove #ERROR! values

In [32]:
error_rows = train[train['free_text'].str.contains("#ERROR!")]
print(error_rows)

      free_text  label_id
1729    #ERROR!         0
1867    #ERROR!         0
3568    #ERROR!         0
10788   #ERROR!         0
11218   #ERROR!         0
11674   #ERROR!         0
15413   #ERROR!         0
16080   #ERROR!         0
18660   #ERROR!         2
20848   #ERROR!         1


In [33]:
train = train[~train['free_text'].str.contains("#ERROR!")]

## Remove duplicates

In [34]:
duplicate_rows = train[train['free_text'].duplicated()]
print(duplicate_rows)

                                        free_text  label_id
225                              reaction thầy ơi         0
442                    đặt khô gà ở đâu v thầy ơi         0
466                              Mệt quá thầy ạ 😞         0
696                                         xin ạ         0
790                     Tại sao phải trả lời thầy         0
...                                           ...       ...
23986  Đình Quang ụa đâu có giải gì bên vn đau ta         0
23992                        Thành Huy Hoàng Tiến         0
24008                     Tự hào thầy Ba gold :))         0
24010                          Lỡ tay thầy ơi :))         0
24022                        Phong Chau cười ỉa 😂         0

[1480 rows x 2 columns]


In [35]:
# Drop duplicates, keep first occurence, make changes directly to the dataframe
train.drop_duplicates(subset='free_text', keep='first', inplace=True)

## Remove outliers
- Some examples have a very big length of characters, not containing helpful information. We will remove them.

In [36]:
num_long_rows = (train['free_text'].str.len() > 500).sum()
print(num_long_rows)

14


In [37]:
train = train[train['free_text'].str.len() <= 500]

## Remove special characters

### URL

In [38]:
# Create a boolean mask for rows that contain a URL
mask = train['free_text'].str.contains(r'http\S+|www\S+', regex=True)

# Use the mask to select a subset of the DataFrame
url_examples = train[mask]

# Print the first few examples
print(url_examples.head())

                                              free_text  label_id
1222  Xem ngay hậu trường cực hiếm của cô Minh Hiếu ...         0
1432  FB chính LinDa: https://www.facebook.com/linda...         0
1839  https://youtu.be/tvyO2B3oEYk thư giãn đã cả nh...         0
2020     @công danh nguyen https://youtu.be/fSypgwW1L_s         0
6891  biệt thự của nude tiger: https://www.google.co...         0


In [39]:
# URL removal
train['free_text'] = train['free_text'].str.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

### Punctuation

In [40]:
train['free_text'] = train['free_text'].str.replace('['+string.punctuation+']', '', regex=True)

### Stopwords
This could led to loss of information, so I will comment this part for now.

In [41]:
# # Load stopwords
# with open('./Stopwords/vietnamese-stopwords.txt', 'r') as f:
#     stopwords = f.read().splitlines()

# # Function to remove stopwords

# def remove_stopwords(text):
#     words = text.split()
#     words = [word for word in words if word not in stopwords]
#     return ' '.join(words)


# # Remove stopwords from 'free_text'
# train['free_text'] = train['free_text'].apply(remove_stopwords)

## Lowercasing

In [42]:
train['free_text'] = train['free_text'].str.lower()

## Emoji processing

In [43]:
# Show emoji rows
def show_emoji():
    mask = train['free_text'].apply(lambda text: any(char in emoji.EMOJI_DATA for char in text))
    # Use the mask to select a subset of the DataFrame
    emoji_rows = train[mask]
    return emoji_rows
# Print the first few rows that contain an emoji
print(show_emoji())

                                               free_text  label_id
0      em được làm fan cứng luôn rồi nè ❤️ reaction q...         0
8                                                    🥰🥰🥰         0
10          được anh ưi lâu rồi không nghe phúc du rap 🤣         0
11                      cắt cho trẻ trâu bớt thui mà 😂😂😂         0
42                                      mệt quá thầy ạ 😞         0
...                                                  ...       ...
23995  thôi xong tóc a chí tôi nay làm vlog đưa djchi...         0
24018               nguyễn loan ý anh ý là thịt a đi e 😝         0
24034  vừa làm chiều nay xong đây  cứ chỗ uy tín mà l...         0
24036                               cuộc sông mưu sinh 🏦         0
24037             lại xui chị trang cho ăn đòn trận bh 🤣         0

[3233 rows x 2 columns]


In [44]:
# Demojize emojis
train['free_text'] = train['free_text'].apply(emoji_vietnamese.demojize)
# Show an example after demojizing
train['free_text'][8]

':mặt cười với 3 trái tim::mặt cười với 3 trái tim::mặt cười với 3 trái tim:'

In [45]:
# Check remaining emojis
print(show_emoji())

                                               free_text  label_id
429                   đỗ thuỷ trinh giông y mạng nhà m 😀         0
623                             gu thời trang gạ địt à 😀         1
667    phương tú ai biết người nghe đâu mà mấy bài đấ...         0
1278        chị cho hỏi  tên tham mỹ viện là vậy chị  😀😀         0
1931                   tuấn bảo chắc bác pro hơn thầy 😀😀         0
...                                                  ...       ...
22508           đã đến lúc thử độ bền của nút haha rồi 😀         0
22939                                             hóng 😀         0
23217                    phúc nhưng phối nhạc hợp với j😀         0
23371                                bắt nộp phạt ngay 😳         0
23475                 phong cách hở rais của người giàu😀         0

[79 rows x 2 columns]


In [46]:
# See list of emojis that have not been processed
text = ' '.join(show_emoji()['free_text'])
emoji.distinct_emoji_list(text)

['😳', '✋', '🐄', '😖', '😀', '😫']

In [47]:
# Create a dictionary of emoji names
emoji_names = {
    '😖': ':mặt bối rối"',
    '😀': ':cười toe toét:',
    '😳': ':mặt ửng đỏ vì ngượng:',
    '✋': ':giơ tay:',
    '🐄': ':con bò:',
    '😫': ':mặt mệt mỏi:'
}
# Replace each emoji with its name
for emoji, name in emoji_names.items():
    train['free_text'] = train['free_text'].str.replace(emoji, name)

In [48]:
# Final check
print(train['free_text'][429])

đỗ thuỷ trinh giông y mạng nhà m :cười toe toét:


## Save cleaned data

In [49]:
# Remove rows with empty text again after processing
train['free_text'].dropna(inplace=True)

In [50]:
train.to_csv('../Dataset_Cleaned/clean_train_vihsd.csv', index=False)